# RAG Langchain with Unstructured PDF
Name      : Willy Santoso<br>
Subject   : RAG Langchain with Unstructured PDF
<br><br>
This code is can be run on Google Colab since this is a Jupyter Notebook file (upload this ipynb to Colab).<br>
If using Google Colab, run this code below to verify Nvidia GPU Driver and install the libraries:

In [ ]:
!nvidia-smi

Fri Aug 30 13:50:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

#### Install PCI utils for Ollama GPU support, libs for Poppler PDF and Tesseract-ocr for image PDF reader

In [ ]:
!sudo apt-get update
!sudo apt-get install -qq -y pciutils
!sudo apt-get install -qq -y libxml2 libxslt1-dev libmagic-dev
!sudo apt-get install -qq -y libnss3 libnss3-dev
!sudo apt-get install -qq -y libcairo2-dev libjpeg-dev libgif-dev
!sudo apt-get install -qq -y cmake libblkid-dev e2fslibs-dev libboost-all-dev libaudit-dev
!sudo apt-get install -qq -y tesseract-ocr
!sudo apt-get install -qq -y libpoppler-dev poppler-utils

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [962 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,261 kB]
Hit:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,498 kB]
Hit:14 ht

In [ ]:
!pip install -q pyngrok

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
!ollama serve > server.log 2>&1 &
!ollama pull nomic-embed-text
!ollama pull llama3.1

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling 970aa74c0a90...   0% ▕▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕▏    0 B/274 MB                  pulling m

In [ ]:
!pip install -q numpy==1.26.4
!pip install -q protobuf==4.25.4
!pip install -q chromadb==0.4.24
!pip install -q onnx==1.16.1
!pip install -q onnxruntime==1.17.1 onnxruntime-gpu==1.17.1
!pip install -q rapidocr-onnxruntime
!pip install -q datasets==2.18.0
!pip install -q pytesseract
!pip install -U -q nltk
!pip install -U -q langchain langchain-core langchain-community
!pip install -U -q langchain-chroma langchain-ollama langchain-huggingface langchainhub langserve langsmith
!pip install -U -q langchain-unstructured unstructured-client unstructured "unstructured[all-docs]" python-magic pydantic lxml pypdf pymupdf
!pip install -U -q ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 6.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 8.1 MB/s eta 0:00

In [ ]:
!wget -O RUU-Ciptaker.pdf "https://ekon.go.id/source/info_sektoral/RUU%20Cipta%20Kerja.pdf"

--2024-08-30 13:55:49--  https://ekon.go.id/source/info_sektoral/RUU%20Cipta%20Kerja.pdf
Resolving ekon.go.id (ekon.go.id)... 103.18.117.53
Connecting to ekon.go.id (ekon.go.id)|103.18.117.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14865737 (14M) [application/pdf]
Saving to: ‘RUU-Ciptaker.pdf’

RUU-Ciptaker.pdf    100%[===================>]  14.18M  7.53MB/s    in 1.9s    

2024-08-30 13:55:52 (7.53 MB/s) - ‘RUU-Ciptaker.pdf’ saved [14865737/14865737]



In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

### Table of Contents
1. Data Preprocessing
2. Retrieval Strategy
3. Model Selection
4. Evaluation Dataset Creation
5. Evaluation
6. Recommendation

### 1. Data Preprocessing
Here we are preparing the required libraries and load the PDF with `UnstructuredPDFLoader`

<b>Import the required libraries</b><br>
- Langchain: main library/tools for RAG
- Chroma: for storing vector databases that generated from Embeddings
- RecursiveCharacterSplitter: use the recursive text splitter for splitting documents
- UnstructurdPDFLoader: for loading unstructured documents like PDFs.
- ChatOllama: use open-source Ollama for Chat LLMs.
- Ollama Embeddings: use open-source Ollama for Embeddings.
- ChatPromptTemplate and PromptTemplate: template for inserting the prompts.
- StrOutputParser: string output parsers.
- RunnablePassthrough: runnable for passing the question inputs from the user.
- MultiQueryRetriever: the main Retrieval method that we want to use.

In [ ]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import PyPDFLoader

from langchain_chroma import Chroma
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

<b>Load the document</b><br>
Let's load the paper

In [ ]:
loader = UnstructuredPDFLoader('./RUU-Ciptaker.pdf')
data = loader.load()

Display head of paper

In [ ]:
print(data[0].page_content[:1000])

RANCANGAN UNDANG-UNDANG REPUBLIK INDONESIA

NOMOR … TAHUN ….

TENTANG

CIPTA KERJA

DENGAN RAHMAT TUHAN YANG MAHA ESA

PRESIDEN REPUBLIK INDONESIA,

Menimbang

:

a. bahwa untuk mewujudkan masyarakat

Indonesia yang sejahtera, adil, dan makmur

berdasarkan Pancasila dan Undang-Undang

Dasar Negara Republik Indonesia Tahun

1945, Negara perlu melakukan berbagai

upaya untuk memenuhi hak warga negara

atas pekerjaan dan penghidupan yang layak

melalui cipta kerja;

b. bahwa dengan cipta kerja diharapkan

mampu menyerap tenaga kerja Indonesia

yang seluas-luasnya di tengah persaingan

yang semakin kompetitif dan

tuntutan

globalisasi ekonomi;

c. bahwa untuk mendukung cipta kerja

diperlukan penyesuaian berbagai aspek

pengaturan

yang

berkaitan

dengan

kemudahan dan perlindungan usaha mikro,

kecil, dan menengah, peningkatan ekosistem

investasi, dan percepatan proyek strategis

nasional,

termasuk

peningkatan

perlindungan dan kesejahteraan pekerja;

1

d. bahwa pengaturan yang berk

<b>Splitting document</b>

For Splitting documents, there is two known method from Langchain:
- `CharacterTextSplitter`: This is a simpler method that splits the text based on a specified character, such as spaces or newlines.
- `RecursiveCharacterTextSplitter`: This method is more advanced and versatile. It attempts to split the text using a series of separators in a hierarchical or recursive manner. For example, it might first try to split the text at paragraph breaks (\n\n), and if the resulting chunks are too large, it then tries to split by single newlines (\n), and if necessary, by spaces, and finally by individual characters.

In [ ]:
# Split and chunk
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(data)

### 2. Retrieval Strategy
For Retrieval Strategy: here we will be using Dense Retrieval (Embedding Vectors) and `MultiQueryRetriever`, it can send multiple queries at one time and one prompt.

<b>Adding Vector Databases to ChromaDB</b><br>
We will using `OllamaEmbeddings` with model: `nomic-embed-text`<br>(open-source Embedding model that claimed surpasses OpenAI `text-embedding-ada-002` and `text-embedding-3-small`, <a href="https://ollama.com/library/nomic-embed-text">[source]</a>)
<br>
Then we need to store the Embedding vectors into Vector Database from ChromaDB

In [ ]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="ciptaker-docs",
    persist_directory="./local-rag-pdf"
)

OllamaEmbeddings: 100%|██████████| 1434/1434 [02:43<00:00,  8.78it/s]


In [ ]:
!zip -r ciptaker-embed.zip ./local-rag-pdf

  adding: local-rag-pdf/ (stored 0%)
  adding: local-rag-pdf/chroma.sqlite3 (deflated 44%)
  adding: local-rag-pdf/8b7a6e41-c0f2-4200-b3dd-2d4a15863cd9/ (stored 0%)
  adding: local-rag-pdf/8b7a6e41-c0f2-4200-b3dd-2d4a15863cd9/data_level0.bin (deflated 10%)
  adding: local-rag-pdf/8b7a6e41-c0f2-4200-b3dd-2d4a15863cd9/link_lists.bin (deflated 88%)
  adding: local-rag-pdf/8b7a6e41-c0f2-4200-b3dd-2d4a15863cd9/index_metadata.pickle (deflated 38%)
  adding: local-rag-pdf/8b7a6e41-c0f2-4200-b3dd-2d4a15863cd9/header.bin (deflated 57%)
  adding: local-rag-pdf/8b7a6e41-c0f2-4200-b3dd-2d4a15863cd9/length.bin (deflated 16%)


### 3. Model Selection
Load the LLM Chat model, we will be using the latest Llama3.1: open-source LLM that improved from Llama3.<br>
For detailed bechmarks and comparisons, here: https://blog.gopenai.com/llama-3-1-vs-llama-3-differences-d3d23e09607f

Why use open-source LLMs? Well, there's several reasons:
- Deployment can be local and anywhere
- No need to pay or subscription services
- Privacy matters, as open-source LLMs runs on local, privacy stays on local machine
- Open-source models has worldwide contributions unlike closed-source is classified contributions
- Open-source has massive support of community developers like HuggingFace, llama.cpp, Ollama, Langchain, etc.

In [ ]:
llm = ChatOllama(model='llama3.1')

Define the Query Prompt Template

In [ ]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

<b>Still about Retrieval Strategy</b>

We will be using `MultiQueryRetriever` for main strategy, as it can send multiple queries at one time and one prompt.

In [ ]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(),
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

Create the Retrieval Chains

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

### 4. Evaluation Dataset Creation
Develop an evaluation dataset that reflects realistic queries researchers might have. Include both simple and complex queries (multi-hop, comparing two things, multiple questions in one prompt) that test the system's ability to retrieve and generate accurate information. Around 20 questions should be enough.

In [ ]:
queries = [
    "What is the main contribution of the Transformer model?",
    "What does the Scaled Dot-Product Attention mechanism do?",
    "How many layers are there in the encoder stack of the Transformer?",
    "Explain the architecture of the Transformer model?",
    "What tasks were used to evaluate the Transformer model?",
    "What are the differences between the encoder and decoder stacks in the Transformer model?",
    "What are the advantages of using self-attention in the Transformer model?",
    "How does the Transformer model handle positional information in sequences?",
    "Explain the regularization techniques used in training the Transformer",
    "How do the training schedules differ between the base and big Transformer models?",
    "Compare the training efficiency of the Transformer with RNN-based models",
    "How does the positional encoding in the Transformer differ from other models?",
    "How does the Transformer handle long-range dependencies compared to convolutional networks?",
    "What is the purpose of the positional encoding in the Transformer?",
    "What hardware was used to train the Transformer models?",
    "How many attention heads are used in the Transformer?",
    "What is the main challenge that self-attention addresses in sequence modeling?",
    "What is the effect of using multi-head attention as shown in the paper?",
    "Describe the key components of the Transformer's encoder and decoder stacks",
    "What training data was used for the Transformer model?",
]
ground_truths = [
    "The Transformer have ability to achieve state-of-the-art results without relying on recurrent neural networks (RNNs) or convolutional layers, relying only on attention mechanisms",
    "The Scaled Dot-Product Attention maps a query and a set of key K and value V pairs to an output, computed as a weighted sum of the values.",
    "The encoder stack consists of 6 identical layers.",
    "The Transformer model consists of an encoder-decoder architecture, with each having a stack of 6 identical layers utilizing self-attention and fully connected feed-forward network.",
    "The Transformer model was evaluated on two machine translation tasks: WMT 2014 English-to-German and WMT 2014 English-to-French.",
    "The encoder consists of two sub-layers: self-attention and a feed-forward network. The decoder includes an additional third sub-layer for encoder-decoder attention.",
    "Self-attention allows for parallelization, reduces path lengths, and enables the model to learn long-range dependencies effectively.",
    "The Transformer uses positional encodings based on sine and cosine functions to inject positional information into the sequence.",
    "Regularization techniques include residual dropout, label smoothing, and weight sharing across embedding layers.",
    "The base model is trained for 12 hours, and the big model for 3.5 days, with corresponding increases in training steps.",
    "The Transformer is significantly more parallelizable and trains faster than recurrent-based models, achieving better and faster performance with less computational resources.",
    "The Transformer uses fixed sine and cosine functions, whereas other models might use learned positional embeddings.",
    "The Transformer’s self-attention mechanism allows it to capture long-range dependencies with a constant number of sequential operations, unlike convolutional networks that are limited to a local window and require stacking multiple layers.",
    "Positional encoding is used to inject information about the relative or absolute position of the tokens in the sequence.",
    "The Transformer models were trained using 8 NVIDIA P100 GPUs.",
    "The Transformer employs 8 parallel attention heads in its multi-head attention mechanism.",
    "Self-attention addresses the challenge of modeling dependencies without requiring their distance in the input or output sequences.",
    "Multi-head attention allows the model to focus on different positions within the sequence, providing a more nuanced understanding by attending to different subspaces.",
    "The encoder consists of two sub-layers: self-attention and position-wise feed-forward networks, while the decoder adds encoder-decoder attention to these components.",
    "The WMT 2014 English-to-German and English-to-French datasets were used for training the Transformer."
]
contexts = []
answers = []

<b>Question no.1</b><br>
Simple Queries with MultiQueryRetriever: "What is the main contribution of the Transformer model?"

In [ ]:
content = []
retrieval = retriever.invoke("What is the main contribution of the Transformer model?")

for item in retrieval:
    content.append(item.page_content)
contexts.append(content)

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]


In [ ]:
answer = chain.invoke("What is the main contribution of the Transformer model?")
answers.append(answer)
answer

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]


'The main contribution of the Transformer model is its ability to achieve state-of-the-art results in machine translation tasks without relying on recurrent neural networks (RNNs) or convolutional layers, instead using only self-attention mechanisms and fully connected feed-forward networks. This allows for more parallelization and faster training times, while still achieving superior performance compared to previous models.'

<b>Question no.2</b><br>
Simple Queries with MultiQueryRetriever: "What does the Scaled Dot-Product Attention mechanism do?"

In [ ]:
content = []
retrieval = retriever.invoke("What does the Scaled Dot-Product Attention mechanism do?")

for item in retrieval:
    content.append(item.page_content)
contexts.append(content)

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]


In [ ]:
answer = chain.invoke("What does the Scaled Dot-Product Attention mechanism do?")
answers.append(answer)
answer

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]


'The Scaled Dot-Product Attention mechanism computes a weighted sum of the values matrix V, where the weights are determined by the softmax function applied to the dot product of the query Q and key K matrices, scaled by the square root of the dimensionality of the keys dk. This is expressed as:\n\nsoftmax(QK^T / sqrt(dk)) * V\n\nThis mechanism allows each position in the input sequence to attend to every other position in the same sequence, and compute a weighted sum of the values at those positions, with the weights determined by the similarity between the query and key vectors.'

<b>Question no.3</b><br>
Simple Queries with MultiQueryRetriever: "How many layers are there in the encoder stack of the Transformer?"

In [ ]:
content = []
retrieval = retriever.invoke("How many layers are there in the encoder stack of the Transformer?")

for item in retrieval:
    content.append(item.page_content)
contexts.append(content)

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.07s/it]


In [ ]:
answer = chain.invoke("How many layers are there in the encoder stack of the Transformer?")
answers.append(answer)
answer

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.07s/it]


'According to the provided text, the encoder stack of the Transformer has 6 identical layers.'

<b>Question no.4</b><br>
Complex Queries with MultiQueryRetriever: "Explain the architecture of the Transformer model?"

In [ ]:
content = []
retrieval = retriever.invoke("Explain the architecture of the Transformer model?")

for item in retrieval:
    content.append(item.page_content)
contexts.append(content)

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]


In [ ]:
answer = chain.invoke("Explain the architecture of the Transformer model?")
answers.append(answer)
answer

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]


"The Transformer model architecture consists of two main components: the encoder and the decoder. Both the encoder and decoder are composed of multiple layers, each containing two sub-layers: a self-attention mechanism and a position-wise fully connected feed-forward network.\n\nIn the encoder, each layer applies self-attention to all positions in the input sequence, allowing each position to attend to all other positions in the sequence. This is done using multi-head attention, which allows the model to jointly attend to information from different representations of the input sequence. The output of this sub-layer is then processed by a position-wise fully connected feed-forward network.\n\nIn the decoder, the first sub-layer uses self-attention on all positions up to and including that position in the decoder's output sequence, followed by another self-attention sub-layer that attends to all positions in the input sequence. This allows the model to attend to information from the enti

<b>Question no.5</b><br>
Simple Queries with MultiQueryRetriever: "What tasks were used to evaluate the Transformer model?"

In [ ]:
content = []
retrieval = retriever.invoke("What tasks were used to evaluate the Transformer model?")

for item in retrieval:
    content.append(item.page_content)
contexts.append(content)

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]


In [ ]:
answer = chain.invoke("What tasks were used to evaluate the Transformer model?")
answers.append(answer)
answer

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.10s/it]


'The Transformer model was evaluated on two machine translation tasks:\n\n1. WMT 2014 English-to-German translation task\n2. WMT 2014 English-to-French translation task\n\nThese tasks were used to assess the performance of the Transformer model in terms of BLEU score, a metric that measures the quality of translations.'

<b>Question no.6</b><br>
Multi-Hop Queries with MultiQueryRetriever: "What are the differences between the encoder and decoder stacks in the Transformer model?"

In [ ]:
content = []
retrieval = retriever.invoke("What are the differences between the encoder and decoder stacks in the Transformer model?")

for item in retrieval:
    content.append(item.page_content)
contexts.append(content)

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]


In [ ]:
answer = chain.invoke("What are the differences between the encoder and decoder stacks in the Transformer model?")
answers.append(answer)
answer

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]


"According to Document(metadata={'source': 'Attention-is-all-you-need.pdf'}, page_content='3 Model Architecture\\n\\nMost competitive neural sequence transduction models have an encoder-decoder structure [5, 2, 29]. Here, the encoder maps an input sequence of symbol representations (x1, ..., xn) to a sequence of continuous representations z = (z1, ..., zn). Given z, the decoder then generates an output sequence (y1, ..., ym) of symbols one element at a time. At each step the model is auto-regressive [9], consuming the previously generated symbols as additional input when generating the next.\\n\\nThe Transformer follows this overall architecture using stacked self-attention and point-wise, fully connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1, respectively.' ), \n\nthe main differences between the encoder and decoder stacks in the Transformer model are:\n\n* The number of sub-layers in each stack: The encoder has two sub-layers per laye

<b>Question no.7</b><br>
Multi-Hop Queries with MultiQueryRetriever: "What are the advantages of using self-attention in the Transformer model?"

In [ ]:
content = []
retrieval = retriever.invoke("What are the advantages of using self-attention in the Transformer model?")

for item in retrieval:
    content.append(item.page_content)
contexts.append(content)


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.94s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.06s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.05s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.10s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]


In [ ]:
answer = chain.invoke("What are the advantages of using self-attention in the Transformer model?")
answers.append(answer)
answer

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.10s/it]


'According to the text, the advantages of using self-attention in the Transformer model include:\n\n* Significantly more parallelization, allowing for faster training times\n* Ability to reach a new state of the art in translation quality after being trained for as little as twelve hours on eight P100 GPUs\n* No need to use sequence-aligned RNNs or convolution, making it a more efficient and scalable model\n\nAdditionally, self-attention is described as yielding more interpretable models, allowing researchers to inspect attention distributions and understand how individual attention heads learn to perform different tasks.'

<b>Question no.8</b><br>
Multi-Hop Queries with MultiQueryRetriever: "How does the Transformer model handle positional information in sequences?"

In [ ]:
content = []
retrieval = retriever.invoke("How does the Transformer model handle positional information in sequences?")

for item in retrieval:
    content.append(item.page_content)
contexts.append(content)


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.92s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.06s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.05s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]


In [ ]:
answer = chain.invoke("How does the Transformer model handle positional information in sequences?")
answers.append(answer)
answer

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.10s/it]


'The Transformer model handles positional information in sequences by injecting positional encodings into the input embeddings at each position. This allows the model to make use of the order of the sequence without relying on recurrence or convolution, which are not used in the Transformer architecture. The positional encodings are added to the input embeddings as a way to provide information about the relative or absolute position of the tokens in the sequence.'

<b>Question no.9</b><br>
Multi-Hop Queries with MultiQueryRetriever: "Explain the regularization techniques used in training the Transformer"

In [ ]:
content = []
retrieval = retriever.invoke("Explain the regularization techniques used in training the Transformer")

for item in retrieval:
    content.append(item.page_content)
contexts.append(content)


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.94s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.03s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.04s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]


In [ ]:
answer = chain.invoke("Explain the regularization techniques used in training the Transformer")
answers.append(answer)
answer

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]


"According to Document(metadata={'source': 'Attention-is-all-you-need.pdf'}, page_content='5.4 Regularization'), the Transformer model employs three types of regularization during training:\n\n1. Residual Dropout: This involves applying dropout [27] to the output of each sub-layer, before it is added to the sub-layer input and normalized. Additionally, dropout is applied to the sums of the embeddings and the positional encodings in both the encoder and decoder stacks.\n2. (Implicitly mentioned as a separate regularization technique) The model also uses dropout with a rate of Pdrop = 0.1 for the base model.\n\nNote: There are no other regularization techniques explicitly mentioned in this context, so it seems that these two are the only ones used in training the Transformer model."

<b>Question no.10</b><br>
Multi-Hop Queries with MultiQueryRetriever: "How do the training schedules differ between the base and big Transformer models?"

In [ ]:
content = []
retrieval = retriever.invoke("How do the training schedules differ between the base and big Transformer models?")

for item in retrieval:
    content.append(item.page_content)
contexts.append(content)


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.65s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.04s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.04s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.10s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.10s/it]


In [ ]:
answer = chain.invoke("How do the training schedules differ between the base and big Transformer models?")
answers.append(answer)
answer

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]


"According to Table 2 in the provided text, the main difference in the training schedules between the base and big Transformer models is that the big model (Transformer (big)) took longer to train than the base model, specifically:\n\n* On the WMT 2014 English-to-German translation task: The base model's training time is not explicitly stated, but the big model took 3.5 days on 8 P100 GPUs.\n* On the WMT 2014 English-to-French translation task: There is no mention of a difference in training schedule between the base and big models.\n\nIt appears that the main difference lies in the computational resources (number of GPUs) used for the big model, which indicates a potential increase in computing power to train the more complex big Transformer model."

<b>Question no.11</b><br>
Comparison Queries with MultiQueryRetriever: "Compare the training efficiency of the Transformer with RNN-based models"

In [ ]:
content = []
retrieval = retriever.invoke("Compare the training efficiency of the Transformer with RNN-based models")

for item in retrieval:
    content.append(item.page_content)
contexts.append(content)


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.96s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.05s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.03s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.07s/it]


In [ ]:
answer = chain.invoke("Compare the training efficiency of the Transformer with RNN-based models")
answers.append(answer)
answer

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]


'The Transformer model achieves significantly faster training compared to architectures based on recurrent or convolutional layers. According to Table 2 in the paper, the Transformer requires only a fraction of the training cost (in terms of FLOPs) compared to other state-of-the-art models such as ByteNet, Deep-Att + PosUnk, GNMT + RL, ConvS2S, MoE, and their respective ensembles. Specifically:\n\n* On the EN-DE translation task, the Transformer requires 2.3 · 10^19 FLOPs, which is significantly less than the other models:\n\t+ ByteNet: 9.6 · 10^18 FLOPs\n\t+ Deep-Att + PosUnk: 2.0 · 10^19 FLOPs (similar to Transformer)\n* On the EN-FR translation task, the Transformer requires 5.1 · 10^19 FLOPs, which is still significantly less than the other models:\n\t+ ByteNet: 9.6 · 10^18 FLOPs\n\t+ Deep-Att + PosUnk: 2.0 · 10^19 FLOPs (similar to Transformer)\n\nThis indicates that the Transformer model can be trained much faster and with less computational resources compared to RNN-based models

<b>Question no.12</b><br>
Comparison Queries with MultiQueryRetriever: "How does the positional encoding in the Transformer differ from other models?"

In [ ]:
content = []
retrieval = retriever.invoke("How does the positional encoding in the Transformer differ from other models?")

for item in retrieval:
    content.append(item.page_content)
contexts.append(content)


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.72s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.04s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.04s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.10s/it]


In [ ]:
answer = chain.invoke("How does the positional encoding in the Transformer differ from other models?")
answers.append(answer)
answer

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]


'The positional encoding in the Transformer differs from other models by being injected into the input embeddings, as opposed to being computed based on the distance between positions. This is stated in Section 3.5 of "Attention-is-all-you-need.pdf", where it says: "To this end, we add \'positional encodings\' to the input embeddings at the input layer".'

<b>Question no.13</b><br>
Comparison Queries with MultiQueryRetriever: "How does the Transformer handle long-range dependencies compared to convolutional networks?"

In [ ]:
content = []
retrieval = retriever.invoke("How does the Transformer handle long-range dependencies compared to convolutional networks?")

for item in retrieval:
    content.append(item.page_content)
contexts.append(content)


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.69s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.06s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.03s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]


In [ ]:
answer = chain.invoke("How does the Transformer handle long-range dependencies compared to convolutional networks?")
answers.append(answer)
answer

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]


'The Transformer handles long-range dependencies by using multi-head attention, which allows it to attend over all positions in the input sequence. This is different from convolutional networks, where dependencies are limited to a local window around each position. By attending over the entire sequence, the Transformer can capture longer-range dependencies and make more informed decisions about which information to use.'

<b>Question no.14</b><br>
Simple Queries with MultiQueryRetriever: "What is the purpose of the positional encoding in the Transformer?"

In [ ]:
content = []
retrieval = retriever.invoke("What is the purpose of the positional encoding in the Transformer?")

for item in retrieval:
    content.append(item.page_content)
contexts.append(content)


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.66s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.04s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.04s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.10s/it]


In [ ]:
answer = chain.invoke("What is the purpose of the positional encoding in the Transformer?")
answers.append(answer)
answer

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]


'The purpose of the positional encoding in the Transformer is to inject information about the relative or absolute position of tokens in the sequence, since the model contains no recurrence and no convolution. This allows the model to make use of the order of the sequence.'

<b>Question no.15</b><br>
Simple Queries with MultiQueryRetriever: "What hardware was used to train the Transformer models?"

In [ ]:
content = []
retrieval = retriever.invoke("What hardware was used to train the Transformer models?")

for item in retrieval:
    content.append(item.page_content)
contexts.append(content)


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.68s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.04s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.05s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]


In [ ]:
answer = chain.invoke("What hardware was used to train the Transformer models?")
answers.append(answer)
answer

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]


'One machine with 8 NVIDIA P100 GPUs.'

<b>Question no.16</b><br>
Simple Queries with MultiQueryRetriever: "How many attention heads are used in the Transformer?"

In [ ]:
content = []
retrieval = retriever.invoke("How many attention heads are used in the Transformer?")

for item in retrieval:
    content.append(item.page_content)
contexts.append(content)


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.68s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.05s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.05s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.07s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]


In [ ]:
answer = chain.invoke("How many attention heads are used in the Transformer?")
answers.append(answer)
answer

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.11s/it]


'According to the text, 8 parallel attention layers, or "heads", are employed in the Transformer. This is stated in section 3.2.3 of the document.'

<b>Question no.17</b><br>
Simple Queries with MultiQueryRetriever: "What is the main challenge that self-attention addresses in sequence modeling?"

In [ ]:
content = []
retrieval = retriever.invoke("What is the main challenge that self-attention addresses in sequence modeling?")

for item in retrieval:
    content.append(item.page_content)
contexts.append(content)


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.67s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.04s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.03s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.11s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.11s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.10s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]

OllamaEmbeddings: 100%|████████

In [ ]:
answer = chain.invoke("What is the main challenge that self-attention addresses in sequence modeling?")
answers.append(answer)
answer

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]


"The main challenge that self-attention addresses in sequence modeling is the need for a mechanism that can relate different positions of a single sequence, without requiring sequential operations or recurrence. This is stated explicitly in Document 8 (metadata={'source': 'Attention-is-all-you-need.pdf'}, page_content='...As noted in Table 1, a self-attention layer connects all positions with a constant number of sequentially executed operations, whereas a recurrent layer requires O(n) sequential operations.')"

<b>Question no.18</b><br>
Complex Queries with MultiQueryRetriever: "What is the effect of using multi-head attention as shown in the paper?"

In [ ]:
content = []
retrieval = retriever.invoke("What is the effect of using multi-head attention as shown in the paper?")

for item in retrieval:
    content.append(item.page_content)
contexts.append(content)


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.64s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.04s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.03s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.07s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]


In [ ]:
answer = chain.invoke("What is the effect of using multi-head attention as shown in the paper?")
answers.append(answer)
answer

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]


'According to the text, the use of multi-head attention has two effects:\n\n1. It reduces the effective resolution due to averaging attention-weighted positions, but this effect is counteracted by the multi-head attention mechanism itself.\n2. As a side benefit, self-attention with multi-head attention can yield more interpretable models, as individual attention heads learn to perform different tasks and may exhibit behavior related to syntactic and semantic structure of sentences.\n\nOverall, the use of multi-head attention appears to be beneficial for improving the performance and interpretability of the model.'

<b>Question no.19</b><br>
Complex Queries with MultiQueryRetriever: "Describe the key components of the Transformer's encoder and decoder stacks"

In [ ]:
content = []
retrieval = retriever.invoke("Describe the key components of the Transformer's encoder and decoder stacks")

for item in retrieval:
    content.append(item.page_content)
contexts.append(content)


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.95s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.04s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.05s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.10s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]


In [ ]:
answer = chain.invoke("Describe the key components of the Transformer's encoder and decoder stacks")
answers.append(answer)
answer

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.10s/it]


"The Transformer's encoder and decoder stacks consist of two sub-layers:\n\n1. **Self-Attention Mechanism**: This mechanism allows the model to attend to all positions in the input sequence simultaneously and weigh their importance. It is applied multiple times with different representations, allowing the model to model complex interactions between different parts of the sequence.\n\n2. **Position-wise Feed-Forward Networks (FFN)**: In addition to self-attention, each layer also includes a position-wise fully connected feed-forward network. This consists of two linear transformations with a ReLU activation in between. The FFN is applied separately to each position and identically across different positions.\n\nBoth sub-layers are followed by residual connections and layer normalization to facilitate the information flow and improve training stability.\n\nIt's worth noting that the self-attention mechanism has some modifications inside, such as masking out (setting to −∞) all values in 

<b>Question no.20</b><br>
Simple Queries with MultiQueryRetriever: "What training data was used for the Transformer model?"

In [ ]:
content = []
retrieval = retriever.invoke("What training data was used for the Transformer model?")

for item in retrieval:
    content.append(item.page_content)
contexts.append(content)


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.92s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.06s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.07s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]


In [ ]:
answer = chain.invoke("What training data was used for the Transformer model?")
answers.append(answer)
answer

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.10s/it]


'The WMT 2014 English-German dataset consisting of about 4.5 million sentence pairs was used for training the Transformer model. For the English-French task, the WMT 2014 English-French dataset consisting of 36M sentences was used.'

### 5. Evaluation
Here we will be using RAGAS evaluation metrics for evaluating RAG results.<br>
There are several RAGAS metrics:<br>
1. Faithfullness: Faithfulness measures how factually consistent the generated answer is with the provided context. It ensures that the claims made in the answer can be directly inferred from the given context. A faithful answer means that all statements in the answer are supported by the context.<br>(Example: If the context states that "Einstein was born on March 14, 1879," an answer saying he was born on March 20, 1879, would have low faithfulness.)<br>
2. Answer Relevancy: This metric assesses how relevant the generated answer is to the original question. It focuses on whether the answer addresses the query directly and avoids unnecessary or unrelated information. Relevance is often measured by generating variations of the question based on the answer and comparing them to the original question using cosine similarity.<br>(Example: For a question asking "What is the capital of France?", an answer stating "Paris is the capital of France" is highly relevant, while an answer discussing French cuisine would score lower.).<br>
3. Context Precision: Context precision evaluates how accurately the retrieved context provides the necessary information to answer the question. It measures whether the relevant information is ranked higher among the retrieved documents, focusing on the signal-to-noise ratio in the retrieval process.<br>(Example: If the context retrieved is directly related to the query without much irrelevant information, the precision is high.).<br>
4. Context Recall: Context recall measures how much of the relevant information from the ground truth is included in the retrieved context. It compares the retrieved context with the expected answer to see if all necessary details are captured.<br>(Example: If the ground truth for a question includes multiple details, and the retrieved context covers most of these, the recall would be high.).<br>
5. Context Entity Recall: This metric is similar to context recall but focuses specifically on entities mentioned in the context. It checks if the retrieved context includes all the entities that are crucial to answering the question.<br>(Example: If a query is about "Albert Einstein," the context entity recall checks if the retrieved documents mention Einstein.).<br>
6. Answer Similarity: Answer similarity assesses how close the generated answer is to the ground truth by comparing the semantic similarity between the two. This metric ensures that even if different words are used, the meaning conveyed is similar.<br>(Example: If the ground truth answer is "Paris is the capital of France," and the generated answer is "The capital of France is Paris," the similarity would be high.).<br>
7. Answer Correctness: This metric evaluates the factual correctness of the generated answer by comparing it to the ground truth. It considers both the factuality and the semantic similarity of the answer to ensure it is correct.<br>(Example: If the ground truth states "Paris is the capital of France," and the answer says "London is the capital of France," the correctness would be low.).

In [ ]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    context_entity_recall,
    answer_similarity,
    answer_correctness,
)
from datasets import Dataset

d = {
    "question": queries,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truths
}
dataset = Dataset.from_dict(d)

In [ ]:
score = evaluate(dataset,
                 metrics=[faithfulness, answer_relevancy, context_precision, context_recall, context_entity_recall, answer_similarity, answer_correctness],
                 llm=ChatOllama(model='llama3.1'),
                 embeddings=OllamaEmbeddings(model='nomic-embed-text'))
score_df = score.to_pandas()
score_df

The evaluation scores can be exported to CSV format if decided

In [ ]:
score_df.to_csv("EvaluationScores.csv", encoding="utf-8", index=False)

### 6. Recommendation
From the Evaluation Dataset Creation result, it seems the strategy needs to be improved. As the answers from LLMs there is correct information but also there is some hallucination and over-typing the messages. Also, the Document PDF Loader needs to be improved too, as another approach (Semi-Structurd PDF Loader <a href="https://github.com/langchain-ai/langchain/blob/master/cookbook/Semi_Structured_RAG.ipynb">from here</a>) will be better preprocessing the documents (but i'm facing issues installing Semi-Structued PDF Loader on my machine, so i used `UnstructuredPDFLoader` instead).<br>
There are also wide of Retrieval Strategies (<a href="https://medium.com/@abhinavkimothi/rag-value-chain-retrieval-strategies-in-information-augmentation-for-large-language-models-3a44845e1e26">as shown on this medium blog</a>) than only Multi-Query Retrieval alone, it is good to explore all of Retrieval technique for comparing and testing RAG results. Finally the RAGAS evaluation is also needs to be improved on open-source LLMs (as i'm facing issue generating the eval scores).